In [1]:
import pandas as pd
import numpy as np

In [2]:
gss = pd.read_csv('/mnt/d/gss_bayes.csv',header=0, index_col=0)

In [3]:
def prob(A: pd.Series) -> float:
    """
    주어진 A의 확률을 계산합니다.
    
    Parameters:
        A (pd.Series): boolean 값을 가진 Series
    
    Returns:
        float: A의 평균 (확률)
    """
    return A.mean()

In [4]:
def conditional(proposition, given):
    """
    조건부확률을 계산.
    given은 이항boolean series, proposition도 이항 boolean series. 
    proposition[given] = proposition[boolean indexing] ; 
    given 이 true 인 행들만이 남고, 그 중 proposition의 데이터를 확인한다. 
    """
    return prob(proposition[given])

- caseid: 응답자 ID
- year: 설문연도
- age: 나이
- sex: 성별(남:1, 여:2)
- polviews: 정치적 성향(1~7, 1:매우 진보적, 7:매우 보수적)
- partyid: 정당소속(0~7, 0:적극적 민주당원, 1:보통 민주당원, 2: 무소속이나 민주당원 지지, 3: 무소속, 4: 무소속이나 공화당 지지, 5: 보통 공화당원, 6: 적극적 공화당원, 7: 다른 정당원)
- Indus10: 산업분야(은행원 6870)

In [5]:
gss

,year,age,sex,polviews,partyid,indus10
caseid,,,,,,
1,1974,21.0,1,4.0,2.0,4970.0
2,1974,41.0,1,5.0,0.0,9160.0
5,1974,58.0,2,6.0,1.0,2670.0
6,1974,30.0,1,5.0,4.0,6870.0
7,1974,48.0,1,5.0,4.0,7860.0
...,...,...,...,...,...,...
2863,2016,57.0,2,1.0,0.0,7490.0
2864,2016,77.0,1,6.0,7.0,3590.0
2865,2016,87.0,2,4.0,5.0,770.0


In [6]:
# 은행원
banker = (gss['indus10'] == 6870)
banker.head()

caseid
1    False
2    False
5    False
6     True
7    False
Name: indus10, dtype: bool

In [7]:
banker.sum()

728

In [8]:
liberal = gss['polviews']<=3
democrat = gss['partyid']<=1
female = gss['sex'] == 2

In [9]:
female

caseid
1       False
2       False
5        True
6       False
7       False
        ...  
2863     True
2864    False
2865     True
2866     True
2867    False
Name: sex, Length: 49290, dtype: bool

In [10]:
liberal

caseid
1       False
2       False
5       False
6       False
7       False
        ...  
2863     True
2864    False
2865    False
2866    False
2867    False
Name: polviews, Length: 49290, dtype: bool

In [11]:
liberal[female]

caseid
5       False
10      False
14      False
15      False
19      False
        ...  
2861    False
2862    False
2863     True
2865    False
2866    False
Name: polviews, Length: 26511, dtype: bool

In [12]:
conditional(liberal,given=female)

0.27581004111500884

# 조건부확률

__~라고 할 때, ~일 확률 == ~ 중, ~ 인 비율__

따라서, "한 응답자가 진보적이라고 했을 때, 이 응답자가 민주당원일 확률은?" 이라는 질문은 다음으로 치환 가능하다.

- "진보적인 응답자 중, 민주당원의 비율"

그리고 또 이 질문은 다음 두 단계로 분리하여 구할 수 있다.

1. 진보적인 응답자의 확률
2. 이 중 민주당원의 비율

이는 A and B와는 다르다. A and B의 경우, "진보적이면서 민주당원인 사람의 비율은?" 으로 나타낸다. 

# 정리 1,2,3

__이 모든 정리는 정리 3, 베이즈 정리로 이어지기 위한 과정__

1. 논리곱을 사용해 조건부확률 구하기

- $P(A|B) = \frac{P(A and B)}{P(B)}$

2. 조건부확률을 사용해 논리곱확률 구하기

- $P(A and B) = P(B) * P(A|B)$

- 정리 1의 양쪽에 P(B)를 곱하면 정리 2가 나온다.

3. 조건부확률(P(A|B) or P(B|A))이 주어졌을 때 원래의 확률을 찾는 방법(베이즈 정리)

- $P(A|B) = \frac{P(A)*P(B|A)}{P(B)}$

- 논리곱은 교환이 가능한데(P(A and B) = P(B and A)) 여기에 정리 2를 양쪽에 적용하면 다음을 얻을 수 있다.

- $P(B)P(A|B) = P(A)P(B|A)$ 

- 이 때 전체를 다시 P(B)로 나눈다면 $P(A|B) = \frac{P(A)*P(B|A)}{P(B)}$ 를 얻게 되는 것.


4. 전체확률의 법칙은 각각의 확률을 더해서 전체 확률을 구하는 방법

- $P(A) = \sum P(B_{i})P(A|B_{i})$

# 연습문제

1. 린다가 여성 은행원일 확률
    1. 린다가 진보 성향의 여성 은행원일 가능성
    2. 린다가 진보 성향의 여성 은행원이며 민주당원일 가능성

2. conditional()을 사용해서 다음 확률을 구하자.
    - 응답자가 민주당원일 경우, 진보성향일 확률은 얼마인가?
    - 응답자가 진보성향일 경우, 민주당원일 확률은 얼마인가?
    - given이 무엇인지 주의하라. 

3. 나이가 30세 이하인 경우를 young, 65세 이상을 old라고 정의한다. conservative 는 정치적 관점에 보수적, 조금 보수적, 매우 보수적이라고 답한 사람들로 정의한다. (polviews>=5)
    - 임의로 고른 응답자가 젊은 진보성향일 확률은?
    - 젊은 사람이 진보적일 확률은 얼마인가?
    - 응답자가 늙고 보수성향일 확률은 얼마인가?
    - 보수성향의 사람들이 늙었을 확률은 얼마인가?

In [14]:
# 1.
print(f'린다가 여성은행원일 가능성:{prob(female & banker)}')

# 1.1
print(f'린다가 진보 성향의 여성은행원일 가능성:{prob(female & liberal & banker)}')

# 1.2
print(f'린다가 진보 성향의 여성은행원이며 민주당원일 가능성:{prob(female & liberal & banker & democrat)}')

린다가 여성은행원일 가능성:0.011381618989653074
린다가 진보 성향의 여성은행원일 가능성:0.002556299452221546
린다가 진보 성향의 여성은행원이며 민주당원일 가능성:0.0012375735443294787


In [15]:
# 2.1 
print(f'응답자가 민주당원일 경우, 진보성향일 확률:{conditional(liberal,given=democrat)}')
# 2.2
print(f'응답자가 진보성향일 경우, 민주당원일 확률:{conditional(democrat,given=liberal)}')

응답자가 민주당원일 경우, 진보성향일 확률:0.3891320002215698
응답자가 진보성향일 경우, 민주당원일 확률:0.5206403320240125


In [17]:
# 3
young = (gss['age']<=30)
old = (gss['age']>=65)
conservative = (gss['polviews'] >=5)

# 3.1

print(f'임의로 고른 응답자가 젊은 진보성향일 확률:{prob(young & liberal)}')

# 3.2

print(f'젊은 사람이 진보적일 확률:{conditional(liberal,given=young)}')

# 3.3

print(f'응답자가 늙고 보수성향일 확률:{prob(old & conservative)}')

# 3.4 

print(f'보수성향의 사람들이 나이가 들었을 확률:{conditional(old,given=conservative)}')

임의로 고른 응답자가 젊은 진보성향일 확률:0.07317914384256441
젊은 사람이 진보적일 확률:0.3377024623162625
응답자가 늙고 보수성향일 확률:0.06701156421180766
보수성향의 사람들이 나이가 들었을 확률:0.19597721609113564
